In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("local[1]") \
                    .appName('sparkEx') \
                    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/18 20:53:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 60101)
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Library/Frameworks/Python.framework/Ver

In [5]:
df = spark.read.csv('/Users/shaytavor/Dropbox/Public/Knowledge Base/Spark/Data/USACities.csv', header = 'True', inferSchema = 'True')

In [6]:
df.count()

200

In [7]:
df.createOrReplaceTempView("Cities")

In [8]:
res = spark.sql("select name, pop2021 from Cities")
res.show()

+-------------+-------+
|         name|pop2021|
+-------------+-------+
|New York City|8230290|
|  Los Angeles|3983540|
|      Chicago|2679080|
|      Houston|2323660|
|      Phoenix|1733630|
| Philadelphia|1585010|
|  San Antonio|1581730|
|    San Diego|1427720|
|       Dallas|1347120|
|       Austin|1011790|
|     San Jose|1009340|
|   Fort Worth| 942323|
| Jacksonville| 929647|
|     Columbus| 913921|
|    Charlotte| 912096|
| Indianapolis| 887232|
|San Francisco| 883255|
|      Seattle| 776555|
|       Denver| 749103|
|   Washington| 714153|
+-------------+-------+
only showing top 20 rows



In [9]:
df.select("*").show()

+----+-------------+----+-------+-------+-------+----------+
|rank|         name|usps|pop2021|pop2010| growth|aland_sqmi|
+----+-------------+----+-------+-------+-------+----------+
|   1|New York City|  NY|8230290|8190210| 0.0049|   300.381|
|   2|  Los Angeles|  CA|3983540|3795510| 0.0495|   468.956|
|   3|      Chicago|  IL|2679080|2697480|-0.0068|   227.369|
|   4|      Houston|  TX|2323660|2100280| 0.1064|   640.194|
|   5|      Phoenix|  AZ|1733630|1449040| 0.1964|   517.673|
|   6| Philadelphia|  PA|1585010|1528280| 0.0371|   134.279|
|   7|  San Antonio|  TX|1581730|1332300| 0.1872|   485.113|
|   8|    San Diego|  CA|1427720|1305910| 0.0933|   325.877|
|   9|       Dallas|  TX|1347120|1200350| 0.1223|   339.736|
|  10|       Austin|  TX|1011790| 806164| 0.2551|   319.939|
|  11|     San Jose|  CA|1009340| 954940|  0.057|   177.808|
|  12|   Fort Worth|  TX| 942323| 748441|  0.259|   345.584|
|  13| Jacksonville|  FL| 929647| 823114| 0.1294|   747.467|
|  14|     Columbus|  OH

In [12]:
df.where(df.usps == 'CA').show()

+----+--------------+----+-------+-------+-------+----------+
|rank|          name|usps|pop2021|pop2010| growth|aland_sqmi|
+----+--------------+----+-------+-------+-------+----------+
|   2|   Los Angeles|  CA|3983540|3795510| 0.0495|   468.956|
|   8|     San Diego|  CA|1427720|1305910| 0.0933|   325.877|
|  11|      San Jose|  CA|1009340| 954940|  0.057|   177.808|
|  17| San Francisco|  CA| 883255| 805505| 0.0965|    46.903|
|  35|        Fresno|  CA| 537100| 497769|  0.079|   114.725|
|  36|    Sacramento|  CA| 525398| 467286| 0.1244|     97.73|
|  43|    Long Beach|  CA| 456154| 462431|-0.0136|    50.696|
|  45|       Oakland|  CA| 440981| 391406| 0.1267|    55.893|
|  51|   Bakersfield|  CA| 389007| 348987| 0.1147|   149.758|
|  55|       Anaheim|  CA| 349699| 336760| 0.0384|     50.32|
|  58|     Riverside|  CA| 334772| 305466| 0.0959|    81.258|
|  59|     Santa Ana|  CA| 333130| 325327|  0.024|    27.361|
|  63|      Stockton|  CA| 314835| 292888| 0.0749|    62.172|
|  68|  

In [22]:
df.groupBy('usps').count().sort('count', ascending = False).show()

+----+-----+
|usps|count|
+----+-----+
|  CA|   39|
|  TX|   26|
|  FL|   13|
|  AZ|   10|
|  NV|    8|
|  VA|    7|
|  NC|    7|
|  TN|    6|
|  CO|    6|
|  IL|    5|
|  WA|    5|
|  OH|    5|
|  LA|    4|
|  AL|    4|
|  NY|    4|
|  GA|    4|
|  KS|    4|
|  NJ|    3|
|  OR|    3|
|  MO|    3|
+----+-----+
only showing top 20 rows



In [23]:
def popDiff(pop2021, pop2010):
    return pop2021 - pop2010

In [29]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType, DoubleType

In [26]:
convertUDF = udf(lambda x, y : popDiff(x, y), IntegerType())

In [28]:
df.select(df.name, convertUDF(df.pop2021, df.pop2010).alias('diff')).show()

+-------------+------+
|         name|  diff|
+-------------+------+
|New York City| 40080|
|  Los Angeles|188030|
|      Chicago|-18400|
|      Houston|223380|
|      Phoenix|284590|
| Philadelphia| 56730|
|  San Antonio|249430|
|    San Diego|121810|
|       Dallas|146770|
|       Austin|205626|
|     San Jose| 54400|
|   Fort Worth|193882|
| Jacksonville|106533|
|     Columbus|122978|
|    Charlotte|173652|
| Indianapolis| 65653|
|San Francisco| 77750|
|      Seattle|165925|
|       Denver|145744|
|   Washington|108927|
+-------------+------+
only showing top 20 rows



In [30]:
def density(pop2021, sqmi):
    return pop2021 / sqmi

In [31]:
densityUDF = udf(lambda x, y: density(x, y), DoubleType())

In [32]:
df.withColumn('density', densityUDF(df.pop2021, df.aland_sqmi)).show()

+----+-------------+----+-------+-------+-------+----------+------------------+
|rank|         name|usps|pop2021|pop2010| growth|aland_sqmi|           density|
+----+-------------+----+-------+-------+-------+----------+------------------+
|   1|New York City|  NY|8230290|8190210| 0.0049|   300.381|27399.502631657797|
|   2|  Los Angeles|  CA|3983540|3795510| 0.0495|   468.956|  8494.48562338471|
|   3|      Chicago|  IL|2679080|2697480|-0.0068|   227.369|11782.960737831454|
|   4|      Houston|  TX|2323660|2100280| 0.1064|   640.194|3629.6185218855535|
|   5|      Phoenix|  AZ|1733630|1449040| 0.1964|   517.673| 3348.890129483284|
|   6| Philadelphia|  PA|1585010|1528280| 0.0371|   134.279|11803.856150254322|
|   7|  San Antonio|  TX|1581730|1332300| 0.1872|   485.113|3260.5392970297644|
|   8|    San Diego|  CA|1427720|1305910| 0.0933|   325.877| 4381.162217646535|
|   9|       Dallas|  TX|1347120|1200350| 0.1223|   339.736| 3965.196505521935|
|  10|       Austin|  TX|1011790| 806164

In [33]:
spark.udf.register('convertUDF', popDiff, IntegerType())

<function __main__.popDiff(pop2021, pop2010)>

In [34]:
df.createOrReplaceTempView('Cities')

In [35]:
spark.sql('select name, convertUDF(pop2021, pop2010) as popDiff from Cities').show()

+-------------+-------+
|         name|popDiff|
+-------------+-------+
|New York City|  40080|
|  Los Angeles| 188030|
|      Chicago| -18400|
|      Houston| 223380|
|      Phoenix| 284590|
| Philadelphia|  56730|
|  San Antonio| 249430|
|    San Diego| 121810|
|       Dallas| 146770|
|       Austin| 205626|
|     San Jose|  54400|
|   Fort Worth| 193882|
| Jacksonville| 106533|
|     Columbus| 122978|
|    Charlotte| 173652|
| Indianapolis|  65653|
|San Francisco|  77750|
|      Seattle| 165925|
|       Denver| 145744|
|   Washington| 108927|
+-------------+-------+
only showing top 20 rows



In [36]:
dfStates = spark.read.csv('/Users/shaytavor/Dropbox/Public/Knowledge Base/Spark/Data/USAStates.csv', header = 'True', inferSchema = 'True')

In [39]:
df.join(dfStates, df.usps == dfStates.Abbreviation).drop('Abbreviation').show()

+----+-------------+----+-------+-------+-------+----------+--------------------+
|rank|         name|usps|pop2021|pop2010| growth|aland_sqmi|               State|
+----+-------------+----+-------+-------+-------+----------+--------------------+
|   1|New York City|  NY|8230290|8190210| 0.0049|   300.381|            New York|
|   2|  Los Angeles|  CA|3983540|3795510| 0.0495|   468.956|          California|
|   3|      Chicago|  IL|2679080|2697480|-0.0068|   227.369|            Illinois|
|   4|      Houston|  TX|2323660|2100280| 0.1064|   640.194|               Texas|
|   5|      Phoenix|  AZ|1733630|1449040| 0.1964|   517.673|             Arizona|
|   6| Philadelphia|  PA|1585010|1528280| 0.0371|   134.279|        Pennsylvania|
|   7|  San Antonio|  TX|1581730|1332300| 0.1872|   485.113|               Texas|
|   8|    San Diego|  CA|1427720|1305910| 0.0933|   325.877|          California|
|   9|       Dallas|  TX|1347120|1200350| 0.1223|   339.736|               Texas|
|  10|       Aus